# On-The-Fly (OTF) Data Reduction

This notebook shows how to use `dysh` to calibrate an OTF observation. This more complex observation is not reduced with one simple command, but is always something in two steps.

The first step is calibration, where the calibrated spectra are written to an SDFITS file. The second step takes these spectra and write
them to a FITS cube. For this there are several third-party tools available, there is nothing in dysh yet to cover this.

In [ ]:
from pathlib import Path
import astropy.units as u
from dysh.util.download import from_url
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data

from dysh.spectra import ScanBlock

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
filename = dysh_data(example='mapping-L/data/TGBT17A_506_11.raw.vegas')    


## Data Loading

Next, we use `GBTFITSLoad` to load the data, and then its `summary` method to inspect its contents.

In [ ]:
sdfits = GBTFITSLoad(filename)

In [ ]:
sdfits.summary()

In this particular observation the OTF scanned over the galaxy NGC6946 in 13 Dec scans , followed by an Off position. Instead of the classical OnOff reduction with `getps`, we can use `getsigref` to refer to scan 27 as the common Off.

## Data Reduction

In [ ]:
ifnum=0        # needs to be 0
plnum=0        # pick 0 and 1 to average
fdnum=0        # fixed
nint = 61      # nunber of integrations per scan

scans = list(range(14,27))
ref = 27

In [ ]:
sb = ScanBlock()

for s in scans:
    print(f"Working on scan {scan}")
    sb1 =  sdfits.getsigref(scan=s, ref=ref, fdnum=fdnum, ifnum=ifnum, plnum=plnum)[0]
    sb.append(sb1)
    
sb.write("otf1.fits", overwrite=True)    #  300 ms

## Gridding

The most commonly used task for this is `gbtgridder`, which does not come with dysh.  Here's a super short blurb how to get it, with
the note that it is important to get the correct release branch.   This was the situation spring 2025, and it may change, so be sure to be in touch with the developers for recommendation.

```
      git clone -b release_3.0  https://github.com/GreenBankObservatory/gbtgridder
      pip install -e gbtgridder
```

After installation, either from the shell, or from the notebook, one can grid

```
      gbtgridder --size 32 32  --channels 500:3500 -o otf1 --clobber --auto otf1.fits
```

this process will create `test2_cube.fits` andd `test2_weight.fits`

We should show the FITS header, and we should plot a spectrum.

